# Strawberry

## Libraries dependency and imports

In [1]:
!pip install strawberry-graphql
!pip install uvicorn[standard]
!pip install fastapi
!pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.3/273.3 kB 107.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 260.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 49.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 216.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 193.1 kB/s eta 0:00:00a 0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.8/428.8 kB 258.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 522.3 kB/s eta 0:00:00a 0:00:01


## Helper Func for App in Notebook

V ukázkách dále bude použit kód, který je specifický pro prostředí jupyter a který tak umožňuje spouštět ukázky přímo v notebooku. Fakticky je kódem vytvořen subproces, který zabezpečuje běh serveru. Identifikace subprocesu je uložena v datové struktuře `servers`. Díky tomu lze identifikovat, zda na požadovaném portu již nějaký server běží a v případě potřeby jej zastavit a spustit nový server.

Po ukončení experimentů se serverem (kódem) je nutné tento server zastavit, aby došlo k uvolnění portu. V případe problémů je možné, že bude nezbytné restartovat jupyter, aby byly porty uvolněny. Je-li spuštěn nový server, aniž by běžící na stejném portu byl ukončen, dojde k chybovému stavu.

```python
assert port in [9991, 9992, 9993, 9994]
```
Slouží k ověření, že požadovaný port je dostupný i z prostředí mimo jupyter. Vzpomeňte si na konfiguraci docker stacku a mapování portů mimo jupyter kontejner.

In [2]:
# Code in this cell is just for (re)starting the API on a Process, and other compatibility stuff with Jupyter cells.
# Just ignore it!
import uvicorn
from multiprocessing import Process

servers = {}

def start_api(app=None, port=9992, runNew=True):
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    assert port in [9991, 9992, 9993, 9994], f'port has unexpected value {port}'
    def run():
        uvicorn.run(app, port=port, host='0.0.0.0', root_path='')    
        
    _api_process = servers.get(port, None)
    if _api_process:
        _api_process.terminate()
        _api_process.join()
        del servers[port]
    
    if runNew:
        assert (not app is None), 'app is None'
        _api_process = Process(target=run, daemon=True)
        _api_process.start()
        servers[port] = _api_process

In [3]:
import os
import asyncio
import multiprocessing
servers = {}

def _start_api(app=None, port=9992, runNew=True):
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    assert port in [9991, 9992, 9993, 9994], f'port has unexpected value {port}'
    
    async def runAsync():
        config = uvicorn.Config(app, port=port, host='0.0.0.0', log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
        #uvicorn.run(app, port=port, host='0.0.0.0', root_path='')    
        
    def withLoop():
        process_name = "[Process %s]" % (os.getpid())
        print("%s Started " % process_name)

        #loop = asyncio.get_event_loop()
        loop = asyncio.new_event_loop()
        try:
            loop.run_until_complete(runAsync())
        except KeyboardInterrupt:
            print("%s Loop interrupted" % process_name)
            loop.stop()

        print("%s terminating" % process_name)
        pass
    
    _api_process = servers.get(port, None)
    if _api_process:
        _api_process.terminate()
        _api_process.join()
        del servers[port]
    
    if runNew:
        assert (not app is None), 'app is None'
        _api_process = multiprocessing.Process(target=withLoop)
        #_api_process = Process(target=run, daemon=True)
        _api_process.start()
        servers[port] = _api_process

In [4]:
!pip install gunicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 83.3 kB/s eta 0:00:00 0:00:01


? # Code in this cell is just for (re)starting the API on a Process, and other compatibility stuff with Jupyter cells.
? # Just ignore it!
import uvicorn
from multiprocessing import Process

async def start_api(app=None, port=9992, runNew=True):
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    assert port in [9991, 9992, 9993, 9994], f'port has unexpected value {port}'

    config = uvicorn.Config(app, port=port, host='0.0.0.0', log_level="info")
    server = uvicorn.Server(config)
    await server.serve()

await start_api(app, port=9992, runNew=True)

In [31]:
#start_api(app, port=9992, runNew=False)

## Hello World in Strawberry

Následující kód je "klasický" hello world, po spuštění otevřete 

http://localhost:31102/gql

Přestože je výše v kódu definovaný port 9992, je nutné si uvědomit, že ve stacku learning je tento port mapován na 31102.
Pokud máte jiný stack, či jiné prostředí, přizpůsobte si url adresu.

Neopomeňte server ukončit `start_api(runNew=False)`

In [5]:
import strawberry
import uuid

@strawberry.type(description="""Type for query root""")
class Query:

    @strawberry.field(description="""Returns a hello""")
    async def say_hello(self, info: strawberry.types.Info, id: strawberry.ID) -> str:
        result = f'Hello {id}'
        return result
    
from strawberry.asgi import GraphQL

graphql_app = GraphQL(
    strawberry.federation.Schema(Query), 
    graphiql = True,
    allow_queries_via_get = True
)

from fastapi import FastAPI
app = FastAPI()
app.mount("/gql", graphql_app)

start_api(app, port=9992, runNew=True)

[Process 23563] Started 


INFO:     Started server process [23563]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9992 (Press CTRL+C to quit)


INFO:     172.20.0.1:45804 - "GET /docs HTTP/1.1" 200 OK
INFO:     172.20.0.1:45804 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     172.20.0.1:45824 - "GET /gql HTTP/1.1" 307 Temporary Redirect
INFO:     172.20.0.1:45824 - "GET / HTTP/1.1" 200 OK
INFO:     172.20.0.1:45824 - "POST / HTTP/1.1" 200 OK
INFO:     172.20.0.1:45866 - "POST / HTTP/1.1" 200 OK
INFO:     172.20.0.1:45878 - "POST / HTTP/1.1" 200 OK
INFO:     172.20.0.1:45918 - "POST / HTTP/1.1" 200 OK
INFO:     172.20.0.1:45938 - "POST / HTTP/1.1" 200 OK
INFO:     172.20.0.1:45938 - "POST / HTTP/1.1" 200 OK


In [4]:
start_api(app, port=9992, runNew=False)

## Strawberry extended

In [9]:
import strawberry
import uuid

@strawberry.federation.type(keys=["id"], description="""Entity representing an user""")
class UserGQLModel:
    @classmethod
    async def resolve_reference(cls, info: strawberry.types.Info, id: strawberry.ID):
        result = {'name': f'John({id})', 'id': id}
        return result
    
    @strawberry.field(description="""user name""")
    async def name(self, info: strawberry.types.Info) -> str:
        result = self['name']
        return result
    
    @strawberry.field(description="""user id""")
    async def id(self, info: strawberry.types.Info) -> str:
        result = self['id']
        return result
    
@strawberry.type(description="""Type for query root""")
class Query:

    @strawberry.field(description="""Returns a hello""")
    async def user_by_id(self, info: strawberry.types.Info, id: strawberry.ID) -> UserGQLModel:
        result = await UserGQLModel.resolve_reference(info, id)
        return result
    
from strawberry.asgi import GraphQL

graphql_app = GraphQL(
    strawberry.federation.Schema(Query), 
    graphiql = True,
    allow_queries_via_get = True
)

from fastapi import FastAPI
app = FastAPI()
app.mount("/gql", graphql_app)

start_api(app, port=9992, runNew=True)

INFO:     Started server process [24332]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9992 (Press CTRL+C to quit)


INFO:     172.20.0.1:47834 - "POST / HTTP/1.1" 200 OK
INFO:     172.20.0.1:47834 - "POST / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [24332]


In [10]:
start_api(app, port=9992, runNew=False)